In [ ]:
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install transformers==4.31
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00

In [ ]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random

In [ ]:
dataset = load_dataset('cnn_dailymail', '3.0.0')
dataset

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
dataset["train"] = dataset["train"].select(range(500))
dataset["validation"] = dataset["validation"].select(range(250))
dataset["test"] = dataset["test"].select(range(100))

dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 250
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 100
    })
})

In [ ]:
def format_instruction(article: str, summary: str):
	return f"""### Instruction:
Summarize the following conversation.

### Input:
{article.strip()}

### Summary:
{summary}
""".strip()

In [ ]:
def generate_instruction_dataset(data_point):

    return {
        "Article": data_point["article"],
        "summary": data_point["highlights"],
        "text": format_instruction(data_point["article"],data_point["highlights"])
    }

In [ ]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset).remove_columns(['id'])
    )

In [ ]:
## APPLYING PREPROCESSING ON WHOLE DATASET
dataset["train"] = process_dataset(dataset["train"])
dataset["test"] = process_dataset(dataset["validation"])
dataset["validation"] = process_dataset(dataset["validation"])
# Select 1000 rows from the training split
train_data = dataset['train'].shuffle(seed=42).select([i for i in range(500)])

# Select 100 rows from the test and validation splits
test_data = dataset['test'].shuffle(seed=42).select([i for i in range(50)])
validation_data = dataset['validation'].shuffle(seed=42).select([i for i in range(50)])

train_data,test_data,validation_data

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

(Dataset({
     features: ['article', 'highlights', 'Article', 'summary', 'text'],
     num_rows: 500
 }),
 Dataset({
     features: ['article', 'highlights', 'Article', 'summary', 'text'],
     num_rows: 50
 }),
 Dataset({
     features: ['article', 'highlights', 'Article', 'summary', 'text'],
     num_rows: 50
 }))

In [ ]:
train_data[10]

{'article': 'DENVER, Colorado -- A Colorado man terrorized by threats after testifying against his daughter\'s abusive boyfriend says he has spent $10,000 on a security system, hired a bodyguard for his son\'s wedding and never leaves home without a .45-caliber handgun strapped to his chest. Keith Reynolds was convicted for witness intimidation after threatening witnesses in his domestic assault case. The man, who asked not to be identified because of the sensitivity of the case, says the state did nothing to protect him after the 1999 conviction of Keith Reynolds for domestic abuse -- even after prosecutors told him a hit had been put on his family. A report in the Denver Post pointed out major problems protecting witnesses in the state of Colorado. Paul Logli, chairman of National District Attorneys Association, told Congress this year that witness intimidation has become "almost epidemic," according to the Denver Post. When asked if prosecutors had made the family aware of Colorado\

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "NousResearch/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


(…)h/Llama-2-7b-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)b-hf/resolve/main/generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

(…)7b-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)lama-2-7b-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
index = 0

article = dataset['test'][index]['article']
summary = dataset['test'][index]['highlights']

prompt = f"""
Summarize the following conversation.

### Input:
{article}

### Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

### Input:
(CNN)This weekend marked the official opening of "Fish in the Dark,"  the Broadway debut of Larry David of "Seinfeld" and "Curb Your Enthusiasm" fame. Of course, the odd conventions of live theater mean that "Fish" wasn't quite fresh-to-table at its premiere: It's been playing for several weeks now to sold-out preview crowds, to build buzz and iron out production kinks. And sold out really means sold out: The show has destroyed box office records for the vintage Cort Theatre and might yet set new ones for a nonmusical despite its limited run. That's because David has a huge built-in audience for his unique brand of humor, which "Curb" fans know to be as abrasive as sea salt and as astringent as lemon, unyielding in the face of social niceties, epic humiliation or even common human decency. It's polarizing, and the poles are

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    # target_modules=["query_key_value"],
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

trainable params: 16777216 || all params: 3517190144 || trainable%: 0.477006226934315


In [ ]:
OUTPUT_DIR = "Llama-2-output"

%load_ext tensorboard
%tensorboard --logdir Llama-2-output/runs

<IPython.core.display.Javascript object>

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=2,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False

In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=validation_data,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
13,1.570500,1.790431
26,1.521300,1.784810
39,1.592700,1.769416
52,1.402300,1.768887


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=62, training_loss=1.627099467862037, metrics={'train_runtime': 3313.4023, 'train_samples_per_second': 0.302, 'train_steps_per_second': 0.019, 'total_flos': 1.6997446236635136e+16, 'train_loss': 1.627099467862037, 'epoch': 1.98})

In [ ]:
peft_model_path="./peft-cnn-summary"

trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

('./peft-cnn-summary/tokenizer_config.json',
 './peft-cnn-summary/special_tokens_map.json',
 './peft-cnn-summary/tokenizer.json')

In [ ]:
from transformers import TextStreamer
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear4bit(
                (lora_dropout): Modul

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_dir = "peft-cnn-summary"

# load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
index = 0

article = dataset['test'][index]['article']
summary = dataset['test'][index]['highlights']

prompt = f"""
Summarize the following conversation.

### Input:
{article}

### Summary:
"""

input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
# output = tokenizer.decode(
#     trained_model.generate(
#         inputs_ids,
#         max_new_tokens=100,
#     )[0],
#     skip_special_tokens=True
# )
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'TRAINED MODEL GENERATED TEXT :\n{output}')

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

### Input:
(CNN)This weekend marked the official opening of "Fish in the Dark,"  the Broadway debut of Larry David of "Seinfeld" and "Curb Your Enthusiasm" fame. Of course, the odd conventions of live theater mean that "Fish" wasn't quite fresh-to-table at its premiere: It's been playing for several weeks now to sold-out preview crowds, to build buzz and iron out production kinks. And sold out really means sold out: The show has destroyed box office records for the vintage Cort Theatre and might yet set new ones for a nonmusical despite its limited run. That's because David has a huge built-in audience for his unique brand of humor, which "Curb" fans know to be as abrasive as sea salt and as astringent as lemon, unyielding in the face of social niceties, epic humiliation or even common human decency. It's polarizing, and the poles are

In [ ]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
)
# Merge LoRA and base model
merged_model = trained_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
!zip -r /content/peft-cnn-summary.zip /content/peft-cnn-summary

  adding: content/peft-cnn-summary/ (stored 0%)
  adding: content/peft-cnn-summary/adapter_config.json (deflated 47%)
  adding: content/peft-cnn-summary/adapter_model.bin (deflated 8%)
  adding: content/peft-cnn-summary/tokenizer_config.json (deflated 67%)
  adding: content/peft-cnn-summary/README.md (deflated 65%)
  adding: content/peft-cnn-summary/tokenizer.json (deflated 74%)
  adding: content/peft-cnn-summary/special_tokens_map.json (deflated 72%)


In [ ]:
!zip -r /content/Llama-2-output.zip /content/Llama-2-output

  adding: content/Llama-2-output/ (stored 0%)
  adding: content/Llama-2-output/checkpoint-62/ (stored 0%)
  adding: content/Llama-2-output/checkpoint-62/adapter_config.json (deflated 47%)
  adding: content/Llama-2-output/checkpoint-62/trainer_state.json (deflated 80%)
  adding: content/Llama-2-output/checkpoint-62/scheduler.pt (deflated 57%)
  adding: content/Llama-2-output/checkpoint-62/rng_state.pth (deflated 25%)
  adding: content/Llama-2-output/checkpoint-62/adapter_model.safetensors (deflated 8%)
  adding: content/Llama-2-output/checkpoint-62/adapter_model.bin (deflated 8%)
  adding: content/Llama-2-output/checkpoint-62/training_args.bin (deflated 50%)
  adding: content/Llama-2-output/checkpoint-62/tokenizer_config.json (deflated 67%)
  adding: content/Llama-2-output/checkpoint-62/README.md (deflated 67%)
  adding: content/Llama-2-output/checkpoint-62/tokenizer.json (deflated 74%)
  adding: content/Llama-2-output/checkpoint-62/optimizer.pt (deflated 9%)
  adding: content/Llama-2-o

In [ ]:
!zip -r /content/merged_model.zip /content/merged_model

  adding: content/merged_model/ (stored 0%)
  adding: content/merged_model/generation_config.json (deflated 32%)
  adding: content/merged_model/config.json (deflated 45%)


##Testing on Trained Model

In [ ]:
!pip install transformers==4.31
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!unzip "/content/peft-cnn-summary.zip" -d "/content/peft-cnn-summary"

Archive:  /content/peft-cnn-summary.zip
  inflating: /content/peft-cnn-summary/peft-cnn-summary/adapter_config.json  
  inflating: /content/peft-cnn-summary/peft-cnn-summary/adapter_model.bin  
  inflating: /content/peft-cnn-summary/peft-cnn-summary/README.md  
  inflating: /content/peft-cnn-summary/peft-cnn-summary/special_tokens_map.json  
  inflating: /content/peft-cnn-summary/peft-cnn-summary/tokenizer.json  
  inflating: /content/peft-cnn-summary/peft-cnn-summary/tokenizer_config.json  


In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
import torch

peft_model_dir = "/content/peft-cnn-summary/peft-cnn-summary/"

# load base LLM model and tokenizer
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

(…)h/Llama-2-7b-hf/resolve/main/config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

(…)b-hf/resolve/main/generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [ ]:
def predict_summary(text):
  prompt = f"""
  Summarize the following conversation.

  ### Input:
  {text}

  """

  input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()
  # output = tokenizer.decode(
  #     trained_model.generate(
  #         inputs_ids,
  #         max_new_tokens=100,
  #     )[0],
  #     skip_special_tokens=True
  # )
  outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
  output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

  dash_line = '-'.join('' for x in range(100))
  print(dash_line)
  print(f'INPUT PROMPT:\n{prompt}')
  print(dash_line)
  print(f'TRAINED MODEL GENERATED TEXT :\n{output}')

In [ ]:
text1 = '''Bruce Jenner will break his silence in a two-hour interview with Diane Sawyer later this month. The former Olympian and reality TV star, 65, will speak in a 'far-ranging' interview with Sawyer for a special edition of '20/20' on Friday April 24, ABC News announced on Monday. The interview comes amid growing speculation about the father-of-six's transition to a woman,Â and follows closely behind his involvement in a deadly car crash in California in February. And while the Kardashian women are known for enjoying center stage, they will not be stealing Bruce's spotlight because they will be in Armenia when the interview airs, according to TMZ. Scroll down for video . Speaking out: Bruce Jenner, pictured on 'Keeping Up with the Kardashians' will speak out in a 'far-ranging' interview with Diane Sawyer later this month, ABC News announced on Monday . Return: Diane Sawyer, who recently mourned the loss of her husband, will return to ABC for the interview . Rumors started swirling around Jenner's gender identity last year, when he emerged from a Beverly Hills clinic with his Adam's apple shaved down. His behavior over the past year also fueled speculation as he began embracing an increasingly female appearance, including growing out his hair, shaving his legs and painting hisÂ nails, while reportedly undergoing hormone therapy. He also split from with his wife of more than two decades, Kris Jenner, with whom he has two daughters, Kyle and Kendall. She filed for divorce in September 2014, citing 'irreconcilable differences'. Reports also emerged over the past week that he has received a breast enhancement. 'Bruce had silicone breast implants put in a few weeks ago,' a source told RadarOnline. 'He went with a smaller implant because he didn't want to look ridiculous.' On Sunday, he was seen walking to his car in Malibu but hid his body beneath a bulky sweatshirt. Out and about: Jenner was pictured walking back to his car in Malibu on the weekend and hiding beneath a large sweatshirt on Sunday, days after reports that he had undergone a breast enhancement . Hiding: He also apparently had painted his nails red when he was seen walking on Sunday . According to Radar, Jenner wants to have all surgeries completed in time to make his on-screen debut as a woman on the fall season of 'Dancing with the Stars'. Jenner is also rumored to be filming a spin-off docu-series about the transitionÂ on E!, although his reps have refused to confirm the claims. While Jenner himself has remained silent about his reported transition, some of his relatives, including step-daughter Kim Kardashian, have spoken about about his 'journey'. 'I guess I'll kind of let everyone be curious and I feel like that's his journey to talk about,' Kim recently told Entertainment Tonight. 'I will say that I think Bruce should tell his story his way. I think everyone goes through things in life and I think that story and what Bruce is going through, I think he'll share whenever the time is right.' Jenner, who won gold in the decathlon at the 1976 Olympics, also made headlines earlier this year for his involvement in a deadly car crash in Malibu. Deadly: In February, Jenner's vehicle, which was pulling a trailer and an ATV (seen right) rear-ended a woman's car (left) and pushed it into the lane of an oncoming Hummer. She died at the scene . By his side: Bruce, pictured with his ex-wife Kris Jenner and four of his step-children (from left) Rob, Kim, Kourtney and Khloe, has received support from his family. Kris filed for divorce from him last year . His Cadillac Escalade, which was pulling a trailer and off-road vehicle, plowed into the back of a Lexus and pushed it into the path of an oncoming HummerÂ on February 7. The Lexus was carrying 69-year-old Kim Howe, who died from chest trauma at the scene. Police sources say Jenner is unlikely to be prosecuted because he wasn't drinking, speeding or texting at the time of the fatal crash. His tell-all interview will also be one of Sawyer's first forays back to TV news following the death of her husband, acclaimed director Mike Nichols, following a heart attack last November. Last September, she left the anchor chair of ABC World News and announced that she planned to focus on specials. In February, she presented 'A Nation of Women Behind Bars', in which she went to prisons across the country to speak with female inmates.'''

In [ ]:
text2='''Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â 'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for space in the overhead lockers, crashing elbows and seat back kicking? Tests conducted by the FAA use planes with a 31 inch pitch, a standard which on some airlines has decreased . Many economy seats on United Airlines have 30 inches of room, while some airlines offer as little as 28 inches . Cynthia Corbertt, a human factors researcher with the Federal Aviation Administration, that it conducts tests on how quickly passengers can leave a plane. But these tests are conducted using planes with 31 inches between each row of seats, a standard which on some airlines has decreased, reported the Detroit News. The distance between two seats from one point on a seat to the same point on the seat behind it is known as the pitch. While most airlines stick to a pitch of 31 inches or above, some fall below this. While United Airlines has 30 inches of space, Gulf Air economy seats have between 29 and 32 inches, Air Asia offers 29 inches and Spirit Airlines offers just 28 inches. British Airways has a seat pitch of 31 inches, while easyJet has 29 inches, Thomson's short haul seat pitch is 28 inches, and Virgin Atlantic's is 30-31.'''

In [ ]:
text2

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for s

In [22]:
text='''DENVER, Colorado -- A Colorado man terrorized by threats after testifying against his daughter\'s abusive boyfriend says he has spent $10,000 on a security system, hired a bodyguard for his son\'s wedding and never leaves home without a .45-caliber handgun strapped to his chest. Keith Reynolds was convicted for witness intimidation after threatening witnesses in his domestic assault case. The man, who asked not to be identified because of the sensitivity of the case, says the state did nothing to protect him after the 1999 conviction of Keith Reynolds for domestic abuse -- even after prosecutors told him a hit had been put on his family. A report in the Denver Post pointed out major problems protecting witnesses in the state of Colorado. Paul Logli, chairman of National District Attorneys Association, told Congress this year that witness intimidation has become "almost epidemic," according to the Denver Post. When asked if prosecutors had made the family aware of Colorado\'s witness protection program, the witness in the Reynolds case said, "All they told us was maybe we should move." "I\'m pretty sure I told one of them -- either him or his wife -- about the program. I can\'t remember if I gave them all the details." the prosecutor assigned to the case, who didn\'t want to be named, told CNN. After Reynolds\' conviction, the witness and his wife saw strange cars parked outside their home. They received phone calls during which the only sound on the other end of the line was a gun being cocked. The couple received a death threat from Reynolds himself through the mail. Watch witness describe fear of being targeted » . Reynolds was then sentenced to ten years in a maximum security prison for witness intimidation. However, it is likely he\'ll be released within five years. No national statistics on crimes against witnesses exist, and minimal research has been conducted on the subject. The latest National Institute of Justice survey on record -- conducted more than a decade ago -- shows that more than half of big city prosecutors consider witness intimidation a major problem. Colorado has $50,000 allocated to its witness protection budget. In contrast, the city of Denver spent almost $100,000 on landscaping last year. The state, on average, spends about $1,000 per witness. That figure supposedly includes moving expenses, rent, and furniture. The federal program spends in excess of $40 million per year on witness protection. One possible reason for the disparity is that witnesses in state cases do not get new identities, as do federal witnesses. "It\'s not designed to be a long-term relocation at the public\'s expense; it\'s a way to ensure the immediate safety of the witnesses," according to Peter Weir, executive director of the Colorado Department of Public Safety. Colorado\'s witness protection program is a "joke," according to The Rev. Leon Kelly, founder and executive director of the Open Door Youth Gang Alternatives. Kelly says the state doesn\'t provide enough incentive for witnesses to come forward. More than a decade ago, a young man named Darryl Givens asked Kelly for advice while deciding whether to testify in a murder case or face prison. Givens chose to testify. A few months later, he was shot twice in the head by men he considered friends. That day continues to haunt Kelly. Rhonda Fields says failures in the witness protection program may have contributed to her son\'s murder. Javad Fields was a college graduate who was engaged to be married. He had plans to move east when a Fourth of July barbecue in 2004 in changed all that. He witnessed the murder of his best friend and subsequently testified against three men suspected in the killing. Javad Fields and his fiancee were gunned down while driving along a suburban street in Aurora, Colorado. The couple, both 22, died instantly. Rhonda Fields says her son was never told about a witness protection program. She confronted prosecutors after his murder. Fields told CNN, "I asked them what happened. Why weren\'t any measures taken to safeguard his life? And I was told he never asked for any protection." Javad\'s mother does not think it was her son\'s responsibility to ask for protection. "I think it\'s the authorities\' responsibility to notify witnesses of the dangers that are involved with being a witness," she said. Field\'s case illustrates glaring weaknesses in the witness protection program. Prosecutors filed an order for protection requesting Javad Fields\' personal information be kept secret, but it wasn\'t signed by a judge until one year later, after defense lawyers had already given the suspects Fields\' personal information, along with crucial trial documents. "I felt like the DA\'s office used my son to win their case but did not take the proper measures to safeguard his life," Rhonda Fields told CNN. District Attorney Carol Chambers maintains the program has improved. She says a notice about witness protection is now attached to every subpoena, ensuring witnesses are aware of their options. That didn\'t start, however, until two years after Javad Fields was killed. E-mail to a friend .'''
text

'DENVER, Colorado -- A Colorado man terrorized by threats after testifying against his daughter\'s abusive boyfriend says he has spent $10,000 on a security system, hired a bodyguard for his son\'s wedding and never leaves home without a .45-caliber handgun strapped to his chest. Keith Reynolds was convicted for witness intimidation after threatening witnesses in his domestic assault case. The man, who asked not to be identified because of the sensitivity of the case, says the state did nothing to protect him after the 1999 conviction of Keith Reynolds for domestic abuse -- even after prosecutors told him a hit had been put on his family. A report in the Denver Post pointed out major problems protecting witnesses in the state of Colorado. Paul Logli, chairman of National District Attorneys Association, told Congress this year that witness intimidation has become "almost epidemic," according to the Denver Post. When asked if prosecutors had made the family aware of Colorado\'s witness p

In [ ]:
predict_summary(text)

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

  Summarize the following conversation.

  ### Input:
  DENVER, Colorado -- A Colorado man terrorized by threats after testifying against his daughter's abusive boyfriend says he has spent $10,000 on a security system, hired a bodyguard for his son's wedding and never leaves home without a .45-caliber handgun strapped to his chest. Keith Reynolds was convicted for witness intimidation after threatening witnesses in his domestic assault case. The man, who asked not to be identified because of the sensitivity of the case, says the state did nothing to protect him after the 1999 conviction of Keith Reynolds for domestic abuse -- even after prosecutors told him a hit had been put on his family. A report in the Denver Post pointed out major problems protecting witnesses in the state of Colorado. Paul Logli, chairman of National District Attorneys Association, told Congress this

#ROUGE and BLEU SCORE

In [ ]:
!pip install rouge_score
!pip install scikit-learn

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=8ad07b76374f6ba2387e42e946355868a7f8fa7923146ba0ef1eee534357776f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
from rouge_score import rouge_scorer

def metrics(text, summary):
  reference = nltk.word_tokenize(text)
  candidate = nltk.word_tokenize(summary)

  # Calculate BLEU score
  bleu_score = sentence_bleu([reference], candidate, smoothing_function=SmoothingFunction().method7)
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(text, summary)

  # Print the BLEU score
  print(f"BLEU Score: {bleu_score}")

    # Print ROUGE scores
  print("ROUGE-1:", scores['rouge1'].fmeasure)
  print("ROUGE-2:", scores['rouge2'].fmeasure)
  print("ROUGE-L:", scores['rougeL'].fmeasure)

In [ ]:
dummy_text = '''LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.'''

dummy_sum = '''Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday . Young actor says he has no plans to fritter his cash away . Radcliffe's earnings from first five Potter films have been held in trust fund .'''

In [ ]:
metrics(dummy_text, dummy_sum)

BLEU Score: 4.500588134214787e-06
ROUGE-1: 0.14682539682539683
ROUGE-2: 0.11952191235059761
ROUGE-L: 0.14682539682539683


In [ ]:
text2 = text2
summary2 = '''Experts say that shrinking space on planes is putting passengers at risk .
  The FAA says it conducts tests on how quickly passengers can leave a plane .
  Some airlines offer as little as 28 inches of space between seats .
  United Airlines has 30 inches of space, while some airlines offer 28 inches .
  British Airways has a seat pitch of 31 inches, while easyJet has'''

In [ ]:
text2

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for s

In [ ]:
metrics(text2, summary2)

BLEU Score: 0.006087800707554291
ROUGE-1: 0.28440366972477066
ROUGE-2: 0.23502304147465436
ROUGE-L: 0.22477064220183487


In [ ]:
import nltk

def entity_matching(text, summary):
    text_tokens = nltk.word_tokenize(text)
    summary_tokens = nltk.word_tokenize(summary)

    text_entities = extract_entities(nltk.chunk.ne_chunk(nltk.pos_tag(text_tokens)))
    summary_entities = extract_entities(nltk.chunk.ne_chunk(nltk.pos_tag(summary_tokens)))
    common_entities = set(text_entities).intersection(set(summary_entities))

    return common_entities

def extract_entities(tree):
    entities = []
    if isinstance(tree, nltk.Tree):
        for child in tree:
            entities.extend(extract_entities(child))
    else:
        entities.append(tree[0])
    return entities

In [ ]:
matching_entities = entity_matching(dummy_text, dummy_sum)

if matching_entities:
    print("Common entities found in text and summary:")
    print(matching_entities)
else:
    print("No common entities found.")

NameError: ignored

In [ ]:
import nltk
import sklearn
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

def accuracy(text, summary):
  matching_entities = entity_matching(text, summary)
  true_positives = len(matching_entities)
  false_positives = len(set(summary.split()) - set(matching_entities))
  actual_entities = list(set(text.split()))
  false_negatives = len(actual_entities) - true_positives

  accuracy = true_positives / (true_positives + false_positives)

  print("True Positives:", true_positives)
  print("False Positives:", false_positives)
  print("False Negatives:", false_negatives)
  print("Accuracy:", accuracy)

In [ ]:
accuracy(dummy_text, dummy_sum)

True Positives: 34
False Positives: 4
False Negatives: 260
Accuracy: 0.8947368421052632


In [ ]:
accuracy(text2, summary2)

True Positives: 45
False Positives: 5
False Negatives: 165
Accuracy: 0.9


#BERT SCORE

In [ ]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
from bert_score import score

hypotheses = [text2]
references = [summary2]
P, R, F1 = score(hypotheses, references, lang="en")

(…)o/roberta-large/resolve/main/config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

(…)co/roberta-large/resolve/main/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

(…)co/roberta-large/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(P, R, F1)

tensor([0.8293]) tensor([0.8856]) tensor([0.8565])


In [ ]:
text2

"Ever noticed how plane seats appear to be getting smaller and smaller? With increasing numbers of people taking to the skies, some experts are questioning if having such packed out planes is putting passengers at risk. They say that the shrinking space on aeroplanes is not only uncomfortable - it's putting our health and safety in danger. More than squabbling over the arm rest, shrinking space on planes putting our health and safety in danger? This week, a U.S consumer advisory group set up by the Department of Transportation said at a public hearing that while the government is happy to set standards for animals flying on planes, it doesn't stipulate a minimum amount of space for humans. 'In a world where animals have more rights to space and food than humans,' said Charlie Leocha, consumer representative on the committee.Â\xa0'It is time that the DOT and FAA take a stand for humane treatment of passengers.' But could crowding on planes lead to more serious issues than fighting for s

In [ ]:
summary2

'Experts say that shrinking space on planes is putting passengers at risk .\n  The FAA says it conducts tests on how quickly passengers can leave a plane .\n  Some airlines offer as little as 28 inches of space between seats .\n  United Airlines has 30 inches of space, while some airlines offer 28 inches .\n  British Airways has a seat pitch of 31 inches, while easyJet has'